In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import logging
import os
import h5py
import matplotlib.pyplot as plt
import torch.optim as optim
import time
import math
import os, sys
import seaborn as sns
import warnings
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
# from MaxMin_Norm import MinMaxNormalization
from torch.utils.data import TensorDataset, DataLoader
sys.path.append('../data_process/')
warnings.filterwarnings("ignore")
import pickle


In [2]:
class Resplus(nn.Module):
    def __init__(self, in_channels, out_channels, lng, lat):
        # 模型的输入是一个四维张量 (B, C, lng, lat)
        super(Resplus, self).__init__()
        self.bn1 = nn.BatchNorm2d(64)
        self.avg_pool = nn.AdaptiveAvgPool2d((16,16))
        self.conv1 = nn.Conv2d(64, 56, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.plus_conv = nn.Conv2d(64,8192,kernel_size=(16,16),padding='valid')
        self.conv2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.conv3 = nn.Conv2d(out_channels, out_channels, 3, 1, 1)
        self.relu = nn.ReLU(inplace = True)
        self.fc3 = nn.Linear(1024, 1024)
        # 分别定义两个批归一化层和卷积层
    def forward(self, x):
#         x1 = F.relu(self.bn1(x))
        x1 = x
        conv1A = self.conv1(x1)
        conv1B = self.avg_pool(x1)
        conv1B = self.plus_conv(conv1B)
        conv1B = conv1B.reshape(-1,8,32,32)
        conv1=  torch.cat((conv1A,conv1B),1)
        conv2 = F.relu(self.bn2(conv1))
        conv2 = self.conv2(conv2)
        out = x+conv2
        return out
class Branch_net1(nn.Module):
    def __init__(self, num_res_unit, input_lenght, flow_channel, grid_heigh, grid_width,in_channels):
        super(Branch_net1, self).__init__()
        self.num_res_unit = num_res_unit
        self.input_lenght = input_lenght
        self.flow_channel = flow_channel
        self.grid_heigh = grid_heigh
        self.grid_width = grid_width
        self.in_channels = in_channels
        # 每个分支网络的首部都是用一个卷积网络将输入的特征维度从低维映射到高维
#         self.branch_net = nn.ModuleList([SE_Block(26)])
        
        self.branch_net = nn.ModuleList([Resplus(64, 64, self.grid_heigh, self.grid_width)])
#         nn.ModuleList([nn.Conv2d(self.in_channels, 80, kernel_size=3, stride=1, padding=1)])
#         self.branch_net.append(nn.Conv2d(self.in_channels, 80, kernel_size=3, stride=1, padding=1))
        # 接下来依次添加多个残差卷积单元
        for i in range(1): 
            self.branch_net.append(Resplus(64, 64, self.grid_heigh, self.grid_width))
#             self.branch_net.append(ResUnit1(80, 80, self.grid_heigh, self.grid_width))
        # 每个分支网络的尾部都是用一个卷积网络将输入的特征维度从高维映射到原本的维度
#         self.branch_net.append(CoordAtt(80,80,32))
#         self.branch_net.append(nn.Conv2d(80, 27, kernel_size=3,stride = 1, padding = 1))
        
    # 分支网络的前向传播
    def forward(self, x):
        for layer in self.branch_net:
            x = layer(x)
        return x 
class Model(nn.Module):
    def __init__(self,
                 lr=0.0005, #模型学习率
                 epoch=20, #训练轮次数
                 batch_size=32, #批训练batch大小
                 c_length=6, #邻近性时间流量特征序列默认长度
                 p_length=1, #周期性时间流量特征序列默认长度
                 t_length=1,  #趋势性时间流量特征序列默认长度
                 external_dim=28, #外部特征的维度
                 grid_heigh=32, #网格图的高度
                 grid_width=32, #网格图的宽度
                 flow_channel=2, #流量种类
                 num_res_unit=2, #设定残差卷积单元的数量
                 in_channels = 6,
                 data_min = -10000,  # 输入数据的最小值默认值
                 data_max = 10000):  # 输入数据的最大值默认值
        super(Model, self).__init__()
        self.epoch = epoch  
        self.lr = lr
        self.batch_size = batch_size
        self.c_length = c_length
        self.p_length = p_length
        self.t_length = t_length
        self.external_dim = external_dim
        self.grid_heigh = grid_heigh
        self.grid_width = grid_width

        self.flow_channel  = flow_channel 
        self.num_res_unit = num_res_unit
        self.logger = logging.getLogger(__name__)
        self.data_min = data_min
        self.data_max = data_max
        self.gpu_available = torch.cuda.is_available()
        if self.gpu_available:  #如果GPU存在，则调用
            self.gpu = torch.device("cuda:0")
        self.backbone_net()
        self.save_path="L%d_C%d_P%d_T%d/"% (self.num_res_unit,self.c_length, self.p_length, self.t_length)
        self.best_mse = 10000

    def backbone_net(self): #创建ST-ResNet的主干网络
        #创建邻近性分支网络 
        self.c_net = Branch_net1(self.num_res_unit,self.c_length,
                     self.flow_channel, self.grid_heigh, self.grid_width,64) 
#         #创建周期性分支网络
#         self.p_net = Branch_net(self.num_res_unit,self.p_length,
#                     self.flow_channel, self.grid_heigh, self.grid_width,80)
# #         #创建趋势性分支网络
#         self.t_net = Branch_net(self.num_res_unit,self.t_length,
#                     self.flow_channel, self.grid_heigh, self.grid_width,80)

        self.conv1 = nn.Conv2d(12, 64, 1, 1, padding='same')
        self.conv2 = nn.Conv2d(2, 64, 1, 1, padding='same')
        self.conv3 = nn.Conv2d(2, 64, 1, 1, padding='same')
        self.conv = nn.Conv2d(193, 64, 1, 1, padding='same')
        self.conv4 = nn.Conv2d(64, 2, 1, 1, padding='same')
        self.bn1 = nn.BatchNorm2d(193)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(0.1)
        # 外部特征映射网络， 将外部特征映射成固定维数的特征向量
        self.ext_net = nn.Sequential(
            nn.Linear(28, 15), 
            nn.ReLU(inplace = True),
            nn.Linear(15,1*self.grid_heigh*self.grid_width))
        # 定义用于三个分支网络输出融合的三个融合矩阵参数
        self.w_c = nn.Parameter(torch.randn(2,32,32), requires_grad=True)
        self.w_p = nn.Parameter(torch.randn(2,32,32), requires_grad=True)
        self.w_t = nn.Parameter(torch.randn(2,32,32), requires_grad=True)
    # 整个网络中的前向传播
    def forward(self, xc, xp, xt, ext):
        xc = self.conv1(xc)
        xp = self.conv2(xp[:,2:4,:,:])
        xt = self.conv3(xt[:,2:4,:,:])
        ext_out = self.ext_net(ext).view([-1, 1,
                                        self.grid_heigh, self.grid_width])
#         print(ext_out.shape)
        X=torch.concat((xc,xp,xt,ext_out),1)
        X= self.conv(X)
        X = self.c_net(X)
#         print(X.shape)
#         X = self.Resplus1(X)
#         X = F.relu(self.bn2(X))
        X = self.dropout(X)
        X = self.conv4(X) 
        X = torch.tanh(X)
        return X
    # 模型的训练启动部分
    def train_model(self, train_loader, val_loader):
        optimizer = optim.Adam(self.parameters(),lr = self.lr)
        loss_func = nn.MSELoss() # 定义模型的优化器和损失函数
        early_stop_threshold = 10 # 设定提前停止阈值
        epoch_count = 0
        start_time = time.time()
        for ep in range(self.epoch):
            loss_list = []
            self.train() # 启动模型训练
            for i, (xc, xp, xt, xe, y) in enumerate(train_loader):
                if self.gpu_available:
                    xc = xc.to(self.gpu)
                    xp = xp.to(self.gpu)
                    xt = xt.to(self.gpu)
                    xe = xe.to(self.gpu)
                    y = y.to(self.gpu)
                ypred = self.forward(xc, xp, xt, xe)
                loss = loss_func(ypred, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            # 模型验证部分，根据验证集上的损失保存最优模型
            for i, (xc, xp, xt, xe, y) in enumerate(val_loader):
                if self.gpu_available:
                    xc = xc.to(self.gpu)
                    xp = xp.to(self.gpu)
                    xt = xt.to(self.gpu)
                    xe = xe.to(self.gpu)
                    y = y.to(self.gpu)
                ypred = self.forward(xc, xp, xt, xe)
                val_loss = loss_func(ypred, y)
                loss_list.append(val_loss.item())
                end_time = time.time() 
            val_mse = np.mean(loss_list)
            print("[%.2fs] ep %d val mse %.6f" %(end_time - start_time, ep, val_mse))
            if val_mse < self.best_mse: #保存当前更优模型参数
#                 self.save_model("DeepSTN+")
                self.best_mse = val_mse
                epoch_count = 0
            else:
                epoch_count = epoch_count +1
#             if epoch_count >= early_stop_threshold:
#                 break  # 当超过提前停止阈值时整个循环结束
            self.eval()
            for i, (xc, xp, xt, xe, y) in enumerate(test_loader):
                if self.gpu_available:
                    xc = xc.to(self.gpu)
                    xp = xp.to(self.gpu)
                    xt = xt.to(self.gpu)
                    xe = xe.to(self.gpu)
                    y = y.to(self.gpu) 
                with torch.no_grad():
                    ypred = self.forward(xc, xp, xt, xe)
                ypred = ypred.cpu().numpy()
                y = y.cpu().numpy()
                if i == 0:
                    ypred1 = ypred
                    y1 = y
                else:
                    y1 = np.concatenate((y1,y),axis=0)
                    ypred1 = np.concatenate((ypred1,ypred),axis=0)
            y = [inverse_transform(d) for d in y1]
            ypred = [inverse_transform(d) for d in ypred1]
            y1 = np.array(y).reshape(2752512)
            ypred1 = np.array(ypred).reshape(2752512)
            print(np.sqrt(metrics.mean_squared_error(ypred1, y1)))
            print(metrics.mean_absolute_error(ypred1, y1))
            LISTRMSE.append(np.sqrt(metrics.mean_squared_error(ypred1, y1)))
            LISTMAE.append(metrics.mean_absolute_error(ypred1, y1))
    # 模型的评估测试集启动部分
    def test_model(self, test_loader):
        rmse_list = []
        mae_list = []
        self.eval() # 启动模型评估（固定BN层参数）
        for i, (xc, xp, xt, xe, y) in enumerate(test_loader):
            if self.gpu_available:
                xc = xc.to(self.gpu)
                xp = xp.to(self.gpu)
                xt = xt.to(self.gpu)
                xe = xe.to(self.gpu)
                y = y.to(self.gpu)
            with torch.no_grad():
                ypred = self.forward(xc, xp, xt, xe)
            # 采用RMSE和MAE作为模型评估的指标
            rmse = ((ypred - y) **2).mean().pow(1/2)
            mae = ((ypred - y).abs()).mean()
#             将RMSE和MAE恢复到规范化之前的尺度
            rmse = rmse * (self.data_max - self.data_min)
            mae = mae * (self.data_max - self.data_min)
            rmse_list.append(rmse.item())
            mae_list.append(mae.item())
            ypred = ypred.cpu().numpy()
            y = y.cpu().numpy()
            if i == 0:
                ypred1 = ypred
                y1 = y
            else:
                y1 = np.concatenate((y1,y),axis=0)
                ypred1 = np.concatenate((ypred1,ypred),axis=0)
        
        mae_test = np.mean(mae_list)
        rmse_test = np.mean(rmse_list)
        return rmse_test, mae_test, ypred1, y1
    
    # 模型保存代码
    def save_model(self, name):
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)
        torch.save(self.state_dict(), self.save_path + name + ".pkl")
    # 读取已保存的模型
    def load_model(self, name):
        if not name.endswith(".pkl"):
            name += ".pkl"
        self.load_state_dict(torch.load(self.save_path + name)) 

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
gpu_available = torch.cuda.is_available()
print(torch.cuda.is_available())
if gpu_available:
    gpu = torch.device("cuda:0")
    print(1)

cuda
True
1


In [1]:
LISTRMSE=[]
LISTMAE=[]
def inverse_transform(X):
    X = (X + 1.) / 2.
    X = 1. * X * (1250 - 0) + 0
    return X
if __name__ == '__main__':
    logging.basicConfig(level=logging.DEBUG,format='%(levelname)s-%(message)s')
    # 为模型中所有超参数赋值
    epochs = 200  # 训练的轮次数
    batch_size = 32 # 训练批次大小
    T = 48  # 一天内的时间片段划分
    c_length = 6  # 临近性序列长度
    p_length = 1  # 周期性序列长度
    t_length = 1  # 趋势性序列长度
    grid_height, grid_width = 32, 32  # 网格图的规格
    flow_channel = 2 # 流量类别数
    lr = 0.0005  # 学习率
    external_dim = 28 #额外特征维度
    num_res_unit = 2  # 卷积残差单元个数

    # 读取训练,验证和测试数据（分别需读入临近性，周期性，趋势性，额外特征数据已经标签数据）
    c_train = np.load('data/c_train.npy')
    p_train = np.load('data/p_train.npy')
    t_train = np.load('data/t_train.npy')
    e_train = np.load('data/e_train.npy')
    w_train = np.load('data/w_train.npy')
    train_y = np.load('data/train_y.npy')
    c_val = np.load('data/c_val.npy')
    p_val = np.load('data/p_val.npy')
    t_val = np.load('data/t_val.npy')
    e_val = np.load('data/e_val.npy')
    w_val = np.load('data/w_val.npy')
    val_y = np.load('data/val_y.npy')
    c_test = np.load('data/c_test.npy')
    p_test = np.load('data/p_test.npy')
    t_test = np.load('data/t_test.npy')
    e_test = np.load('data/e_test.npy')
    w_test = np.load('data/w_test.npy')
    test_y = np.load('data/test_y.npy')
    # 读取已保存的归一化数据转换器
    with open ('../data_process/data/preprocessing.pkl', 'rb') as f:
     scale = pickle.load(f)  
#     print(c_train.shape,p_train.shape,t_train.shape,P.shape,C1.shape)
    # 将训练集/验证集/测试集都整理成pytorch的输入格式
    train_set = TensorDataset(torch.Tensor(c_train),torch.Tensor(p_train),torch.Tensor(t_train), 
            torch.Tensor(w_train), torch.Tensor(train_y))
    val_set = TensorDataset(torch.Tensor(c_val),torch.Tensor(p_val),torch.Tensor(t_val),
           torch.Tensor(w_val), torch.Tensor(val_y))
    test_set = TensorDataset(torch.Tensor(c_test),torch.Tensor(p_test),torch.Tensor(t_test),
           torch.Tensor(w_test), torch.Tensor(test_y))

    # 装载训练集/验证集/测试集数据
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle = True,drop_last=False)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle = False,drop_last = False)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle = False,drop_last = False)
    # 定义模型
    net = Model(lr = lr, epoch = epochs, batch_size = batch_size, 
               c_length = c_length, p_length = p_length, t_length = t_length, 
               external_dim = external_dim, grid_heigh = grid_height, 
               grid_width = grid_width, flow_channel = flow_channel, 
               num_res_unit = num_res_unit, data_min = scale._min, data_max = scale._max)

    if gpu_available:
        net = net.to(gpu)
#     # 训练模型
    net.train_model(train_loader, val_loader)
    print('test model....')
    net.load_model("DeepSTN+") # 读取最优模型
    rmse, mae,ypred,y  = net.test_model(test_loader) #用验证集上最好的模型进行测试评估
    y = [inverse_transform(d) for d in y]
    ypred = [inverse_transform(d) for d in ypred]
    y1 = np.array(y).reshape(2752512)
    ypred1 = np.array(ypred).reshape(2752512)
    print('RMSE', np.sqrt(metrics.mean_squared_error(ypred1, y1)))
    print('MAE', metrics.mean_absolute_error(ypred1, y1))

NameError: name 'logging' is not defined